In [1]:
# TestPath = '/content/drive/My Drive/Test'
# OutputFile = '/content/drive/My Drive/Output.txt'
TestPath = "./Test"
OutputFile = "./Output.txt"


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import torchvision
from torchvision import transforms as T
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import Image
import os
from skimage import io
import time
from sklearn.model_selection import train_test_split
from collections import Counter
import copy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from skimage.transform import resize as io_resize
import cv2
import json
import datetime
import pickle as pkl

In [4]:
img_files = [os.path.join(TestPath, name) for name in os.listdir(TestPath)]
RESIZE = (400, 300)


In [5]:
class T1Dataset(torch.utils.data.Dataset):
    """
    This is to use with array of files and array of labels as an input
    """

    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = io.imread(self.X[idx])
        if self.transform is not None:
            image = self.transform(image)
        if self.y is None:
            return (image, idx)
        return (image, self.y[idx])

In [6]:
transformer = T.Compose(
    [
        T.ToPILImage(),
        T.ToTensor(),
        T.Resize(RESIZE),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

In [7]:
dataset = T1Dataset(img_files, None, transformer)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
def predict(model, dataloader):
    model.eval()
    predictions = []

    for inputs,_ in tqdm(dataloader, desc="Testing"):
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        predictions.append(preds.item())

    return predictions


In [10]:
model = pkl.load(open("./DeepModel.pkl", "rb"))
predictions = predict(model, loader)
print(predictions)

with open(OutputFile, "w") as f:
    for pred in predictions:
        f.write(str(pred) + "\n")


Testing:   0%|          | 0/21 [00:00<?, ?it/s]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
